In [3]:
import pandas as pd
from tvDatafeed import TvDatafeed, Interval
import os

# Inicializar o TvDatafeed


# Credenciais do TradingView
username = os.getenv('TRADINGVIEW_USERNAME')
password = os.getenv('TRADINGVIEW_PASSWORD')

tv = TvDatafeed(username, password)

# index
CEAB3 = tv.get_hist(symbol='CEAB3',exchange='BMFBOVESPA',interval=Interval.in_1_hour,n_bars=1000)

print(CEAB3)

ERROR:tvDatafeed.main:error while signin


                               symbol   open   high    low  close    volume
datetime                                                                   
2023-10-23 17:00:00  BMFBOVESPA:CEAB3   5.06   5.11   5.04   5.11  181700.0
2023-10-24 10:00:00  BMFBOVESPA:CEAB3   5.11   5.23   5.10   5.18  681500.0
2023-10-24 11:00:00  BMFBOVESPA:CEAB3   5.18   5.20   5.12   5.14  307300.0
2023-10-24 12:00:00  BMFBOVESPA:CEAB3   5.14   5.16   5.10   5.13  325900.0
2023-10-24 13:00:00  BMFBOVESPA:CEAB3   5.13   5.14   5.04   5.07  237800.0
...                               ...    ...    ...    ...    ...       ...
2024-04-25 15:00:00  BMFBOVESPA:CEAB3  10.69  10.78  10.64  10.77  271800.0
2024-04-25 16:00:00  BMFBOVESPA:CEAB3  10.76  10.92  10.73  10.87  613100.0
2024-04-25 17:00:00  BMFBOVESPA:CEAB3  10.84  10.84  10.74  10.74  126400.0
2024-04-26 10:00:00  BMFBOVESPA:CEAB3  10.97  11.42  10.90  11.42  911500.0
2024-04-26 11:00:00  BMFBOVESPA:CEAB3  11.43  11.52  11.28  11.50  728700.0

[1000 rows 

In [8]:
import pandas as pd
import numpy as np
from tvDatafeed import TvDatafeed, Interval
import os
import pandas_ta

# Credenciais do TradingView
username = os.getenv('TRADINGVIEW_USERNAME')
password = os.getenv('TRADINGVIEW_PASSWORD')
tv = TvDatafeed(username, password)

## Calcula a inclinação da média móvel de 80 períodos comparando o ultimo valor da série com o de X barras antes
def moving_average_slope(mme_series, slope_window=40):
    if len(mme_series) < slope_window:
        return 0  # Evita valores NaN se não tiverem dados suficientes
    # Calcula o slope baseado nos últimos pontos da MME
    raw_slope = (mme_series.iloc[-1] - mme_series.iloc[-slope_window]) / slope_window
    return np.tanh(raw_slope) # Usa a função da tangente hiperbólica para termos valores entre -1 e 1;

## Calculo do estocástico usando pandas_ta; STOCHd é o estocástico lento (suavizado por uma média móvel de 3)
## STOCHk é o estocástico; k é a quantidade de períodos usado pro cálculo do estocástico; 
## smooth_k é a qtd de periódos da média móvel q suaviza o estocástico (%K)
## d é qtd de periodos da  média móvel que suaviza o estocástico lento (%D) [sim, suaviza 2 vezes]
def calculate_stochastic(data):
    if {'high', 'low', 'close'}.issubset(data.columns):
        stoch = data.ta.stoch(high='high', low='low', close='close', k=14, d=3, smooth_k=3)
        return stoch['STOCHk_14_3_3'], stoch['STOCHd_14_3_3']
    else:
        return None, None

def calculate_adx(data, period=14):
    if {'high', 'low', 'close'}.issubset(data.columns):
        adx = data.ta.adx(high='high', low='low', close='close', length=period)
        return adx['ADX_' + str(period)]
    else:
        return None    
    
# Função que analisa os tickers 
def analyze_ticker(ticker):
    try:
        data = tv.get_hist(symbol=ticker, exchange="BMFBOVESPA", interval=Interval.in_1_hour, n_bars=100)
        if data is None or data.empty:
            return None

        # Calcula a MME de 80 períodos, inclinação da MME, estocástico e ADX
        data['MME_80'] = data['close'].ewm(span=80, adjust=False).mean()
        data['MME_Slope'] = moving_average_slope(data['MME_80'])
        data['STOCHk'], data['STOCHd'] = calculate_stochastic(data)
        data['ADX'] = calculate_adx(data)
        
        # Cria uma coluna para verificar se o preço atual está acima/abaixo da MME_80
        data['Price_Above_MME_80'] = data['close'] > data['MME_80']
        data['Price_Below_MME_80'] = data['close'] < data['MME_80']


        most_recent_signal = None
        # Se estocástico lento é maior que 80, preço está abaixo da MME80 e MME80 está inclinada para baixo, com ADX maior que 25, sinal de venda
        # Se estocástico lento é menor que 20, preço está acima da MME80 e MME80 está inclinada para cima, com ADX maior que 25, sinal de compra 
        for i in range(1, len(data)):  # Começa do segundo elemento para acessar o anterior
            row = data.iloc[i]
            prev_row = data.iloc[i-1]  # Acessa a linha anterior para verificar cruzamento

            if row['STOCHd'] < 20 and row['close'] > row['MME_80'] and row['ADX'] > 25:
                signal = 'Sobrevendido/Tendência de Alta'
                date_signal = data.index[i]
                most_recent_signal = {
                    'Ticker': ticker,
                    'Date': date_signal.strftime('%Y-%m-%d %H:%M:%S'),
                    'Close_Price': row['close'],
                    'STOCHk': row['STOCHk'],
                    'STOCHd': row['STOCHd'],
                    'MME_80': row['MME_80'],
                    'MME_Slope': row['MME_Slope'],
                    'ADX': row['ADX'],
                    'Signal': signal
                }
                break
            elif row['STOCHd'] > 80  and row['close'] < row['MME_80'] and row['ADX'] > 25:
                signal = 'Sobrecomprado/Tendência de Baixa'
                date_signal = data.index[i]
                most_recent_signal = {
                    'Ticker': ticker,
                    'Date': date_signal.strftime('%Y-%m-%d %H:%M:%S'),
                    'Close_Price': row['close'],
                    'STOCHk': row['STOCHk'],
                    'STOCHd': row['STOCHd'],
                    'MME_80': row['MME_80'],
                    'MME_Slope': row['MME_Slope'],
                    'ADX': row['ADX'],
                    'Signal': signal
                }
                break

    except Exception as e:
        print(f"Error processing {ticker}: {str(e)}")
    return most_recent_signal

def process_tickers(filename):
    tickers = pd.read_excel(filename)['Ticker'].tolist()
    results_df = pd.DataFrame()  # Inicia um DataFrame vazio
    for ticker in tickers:
        result = analyze_ticker(ticker)
        if result:
            # Cria um DataFrame temporário para o resultado atual
            temp_df = pd.DataFrame([result])
            results_df = pd.concat([temp_df, results_df], ignore_index=True)
            print(f"Processed: {result['Ticker']} at {result['Date']} - {result['Signal']}")
            
    # Ordena o DataFrame pela coluna 'Date' de forma decrescente
    results_df['Date'] = pd.to_datetime(results_df['Date'])  # Garante que 'Date' seja tratado como datetime
    results_df.sort_values(by='Date', ascending=False, inplace=True)            

    return results_df

# Carregar tickers e analisar
file_path = 'tickers_IBOV.xlsx'
result_df = process_tickers(file_path)

# Exibir resultados finais
from IPython.display import display
display(result_df)


ERROR:tvDatafeed.main:error while signin


Processed: PETR3 at 2024-05-03 13:00:00 - Sobrevendido/Tendência de Alta
Processed: ELET3 at 2024-04-25 16:00:00 - Sobrecomprado/Tendência de Baixa
Processed: BBDC4 at 2024-05-02 15:00:00 - Sobrevendido/Tendência de Alta


ERROR:tvDatafeed.main:Connection is already closed.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


Processed: PRIO3 at 2024-04-24 10:00:00 - Sobrecomprado/Tendência de Baixa
Processed: UGPA3 at 2024-05-02 10:00:00 - Sobrevendido/Tendência de Alta
Processed: RAIL3 at 2024-04-26 13:00:00 - Sobrecomprado/Tendência de Baixa
Processed: SBSP3 at 2024-04-23 11:00:00 - Sobrevendido/Tendência de Alta
Processed: VBBR3 at 2024-04-30 13:00:00 - Sobrevendido/Tendência de Alta
Processed: GGBR4 at 2024-04-23 14:00:00 - Sobrevendido/Tendência de Alta
Processed: BBSE3 at 2024-05-06 10:00:00 - Sobrecomprado/Tendência de Baixa


ERROR:tvDatafeed.main:Connection is already closed.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol
ERROR:tvDatafeed.main:Connection is already closed.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


Processed: ASAI3 at 2024-04-24 13:00:00 - Sobrevendido/Tendência de Alta
Processed: HAPV3 at 2024-04-30 13:00:00 - Sobrevendido/Tendência de Alta


ERROR:tvDatafeed.main:Connection is already closed.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


Processed: EMBR3 at 2024-05-02 14:00:00 - Sobrevendido/Tendência de Alta
Processed: TOTS3 at 2024-04-24 11:00:00 - Sobrevendido/Tendência de Alta
Processed: KLBN11 at 2024-05-02 16:00:00 - Sobrecomprado/Tendência de Baixa
Processed: TIMS3 at 2024-04-23 13:00:00 - Sobrevendido/Tendência de Alta
Processed: ELET6 at 2024-04-25 16:00:00 - Sobrecomprado/Tendência de Baixa
Processed: TRPL4 at 2024-04-25 13:00:00 - Sobrecomprado/Tendência de Baixa
Processed: CMIN3 at 2024-04-23 10:00:00 - Sobrevendido/Tendência de Alta
Processed: CPFE3 at 2024-04-29 10:00:00 - Sobrevendido/Tendência de Alta
Processed: GOAU4 at 2024-04-26 14:00:00 - Sobrecomprado/Tendência de Baixa
Processed: RECV3 at 2024-04-22 10:00:00 - Sobrecomprado/Tendência de Baixa
Processed: YDUQ3 at 2024-04-30 12:00:00 - Sobrevendido/Tendência de Alta
Processed: BRAP4 at 2024-04-30 10:00:00 - Sobrecomprado/Tendência de Baixa
Processed: IGTI11 at 2024-04-23 16:00:00 - Sobrecomprado/Tendência de Baixa
Processed: BRKM5 at 2024-04-22 16:0

,Ticker,Date,Close_Price,STOCHk,STOCHd,MME_80,MME_Slope,ADX,Signal
3,BEEF3,2024-05-06 10:00:00,6.030000,61.135802,81.206884,6.112469,0.001268,25.485813,Sobrecomprado/Tendência de Baixa
23,BBSE3,2024-05-06 10:00:00,32.290000,71.029635,85.511414,32.503481,-0.003933,33.225469,Sobrecomprado/Tendência de Baixa
32,PETR3,2024-05-03 13:00:00,42.050000,19.825708,19.659342,41.327072,0.035492,35.205820,Sobrevendido/Tendência de Alta
18,KLBN11,2024-05-02 16:00:00,23.210000,84.800283,81.925298,23.500129,-0.012081,31.548786,Sobrecomprado/Tendência de Baixa
30,BBDC4,2024-05-02 15:00:00,13.853841,17.708333,14.409722,13.836313,0.002177,37.835298,Sobrevendido/Tendência de Alta
20,EMBR3,2024-05-02 14:00:00,32.910000,11.100175,14.262739,32.399322,0.027302,40.181661,Sobrevendido/Tendência de Alta
28,UGPA3,2024-05-02 10:00:00,26.590000,34.294039,16.965714,26.408828,0.001065,25.679270,Sobrevendido/Tendência de Alta
4,MRVE3,2024-04-30 17:00:00,6.670000,13.730159,18.804855,6.612428,0.007338,27.772923,Sobrevendido/Tendência de Alta
25,VBBR3,2024-04-30 13:00:00,23.370000,16.585419,19.843047,23.336299,0.007733,32.514592,Sobrevendido/Tendência de Alta
21,HAPV3,2024-04-30 13:00:00,3.760000,6.250000,11.574074,3.680301,0.002690,38.413727,Sobrevendido/Tendência de Alta
